<a href="https://colab.research.google.com/github/YuvaShnee/AI-Powered-Automation-Workflow-Agent/blob/main/Assignment_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run once
!pip install --upgrade pip
!pip install pdfplumber openai pandas openpyxl tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 70.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pdfplumber]


In [2]:
from google.colab import files
uploaded = files.upload()


Saving Data Input.pdf to Data Input.pdf


In [16]:
##step : 3
##Read the PDF text with pdfplumber

import pdfplumber
def extract_pdf_text(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text() or ""
            text += f"\n\n--- PAGE {i} ---\n" + page_text
    return text

pdf_path = "Data Input.pdf"  # or the Drive path if mounted
raw_text = extract_pdf_text(pdf_path)
print(raw_text[:1200])  # show the first 1200 chars to inspect





--- PAGE 1 ---
Vijay Kumar was born on March 15, 1989, in Jaipur, Rajasthan, making him 35 years old as of 2024.
His birthdate is formatted as 1989-03-15 in ISO format for easy parsing, while his age serves as a
key demographic marker for analytical purposes. Born and raised in the Pink City of India, his
birthplace provides valuable regional profiling context, and his O+ blood group is noted for
emergency contact purposes. As an Indian national, his citizenship status is important for
understanding his work authorization and visa requirements across different employment
opportunities.
Vijay's professional journey began on July 1, 2012, when he joined his first company as a Junior
Developer with an annual salary of 350,000 INR. His career progression shows remarkable growth,
with his current role at Resse Analytics beginning on June 15, 2021, where he serves as a Senior
Data Engineer earning 2,800,000 INR annually. Before this position, he worked at LakeCorp
Solutions from February 1

In [17]:
##step : 4
with open("extracted_text.txt", "w", encoding="utf-8") as f:
    f.write(raw_text)
print("Saved extracted_text.txt")


Saved extracted_text.txt


In [18]:
#step:5
##Configure OpenAI (or set provider)
#If you will use OpenAI: set API key
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter OPENAI_API_KEY (will be hidden): ")


Enter OPENAI_API_KEY (will be hidden): ··········


In [19]:
##step : 6
##LLM call & strict prompt template (this returns JSON)
import openai
import json
import math
from typing import List

# Initialize the OpenAI client
client = openai.OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# Basic helper to split text into chunks to avoid token limit issues
def chunk_text(text, max_tokens=1500, chunk_overlap=200, tokenizer=None):
    # naive chunking by characters but respecting overlap
    # Use tokenizer if you want exact token limits (tiktoken). Here we use characters for simplicity.
    approx_chars = int(max_tokens * 4)  # rough characters per token
    chunks = []
    start = 0
    length = len(text)
    while start < length:
        end = start + approx_chars
        chunk = text[start:end]
        if start != 0:
            # add overlap
            start = end - chunk_overlap
        else:
            start = end
        chunks.append(chunk)
    return chunks

# The central prompt template (very explicit)
PROMPT_INSTRUCTIONS = """
You will be given a block of text extracted from a PDF. DO NOT invent new information,
do NOT paraphrase. Your job is to extract 100% of the content into a JSON array of objects,
where each object has EXACTLY these keys: "Key", "Value", "Comments".

Rules:
1) Identify meaningful keys from the text semantically — DO NOT use a pre-defined key list.
2) Each "Value" must be a short machine-friendly value or the exact snippet from the text.
3) If a sentence or phrase doesn't fit neatly as a key:value pair, put the original sentence in the "Comments" field and set "Key" to an indicative label (e.g. "Context: ...") and "Value" to "" if necessary.
4) Preserve original wording for content in both Value and Comments; do not change sentence structure.
5) For multi-line structures, include the full original lines in Value or Comments (maintain newlines where necessary).
6) Return only JSON — NO explanation, NO surrounding text.

Output format example:
[
  {"Key": "Full Name", "Value": "Vijay Kumar", "Comments": "Original sentence or additional context here."},
  ...
]
Now process the input text below and return JSON.
"""

def call_llm_for_json(blocks: List[str], model="gpt-4o-mini", temperature=0.0):
    # We will accumulate outputs from each chunk, but the LLM MUST return JSON per chunk.
    results = []
    for i, block in enumerate(blocks, start=1):
        system = PROMPT_INSTRUCTIONS + "\n\n=== INPUT TEXT START ===\n" + block + "\n\n=== INPUT TEXT END ===\n"
        # Use ChatCompletion
        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role":"system", "content": "You are a strict JSON-only extractor."},
                {"role":"user", "content": system}
            ],
            temperature=temperature,
            max_tokens=3000
        )
        content = resp.choices[0].message.content.strip()
        # Parse JSON safely
        try:
            parsed = json.loads(content)
            if not isinstance(parsed, list):
                raise ValueError("Expected a list")
        except Exception as e:
            print(f"Error parsing JSON from chunk {i}: {e}")
            print("Raw model output (first 400 chars):", content[:400])
            raise
        results.extend(parsed)
    return results

# Chunk and call
chunks = chunk_text(raw_text, max_tokens=1500, chunk_overlap=300)
extracted_list = call_llm_for_json(chunks, model="gpt-4o-mini", temperature=0.0)
print(f"Extracted {len(extracted_list)} key:value entries.")

Extracted 39 key:value entries.


In [20]:
##step : 7
# Quick verification: ensure each non-empty line is present in some Value or Comments
extracted_text_blobs = []
for obj in extracted_list:
    for key in ("Key","Value","Comments"):
        if obj.get(key):
            extracted_text_blobs.append(str(obj.get(key)))

joined_extracted = "\n".join(extracted_text_blobs)

missing_lines = []
for line in raw_text.splitlines():
    line = line.strip()
    if not line:
        continue
    if line not in joined_extracted:
        missing_lines.append(line)

print(f"Total non-empty lines in PDF: {len([l for l in raw_text.splitlines() if l.strip()])}")
print("Number of lines not found verbatim in outputs (best-effort):", len(missing_lines))
if len(missing_lines)>0:
    print("Sample missing lines (up to 10):")
    for s in missing_lines[:10]:
        print("-", s)
else:
    print("All lines appear verbatim in Value/Comments — good!")


Total non-empty lines in PDF: 35
Number of lines not found verbatim in outputs (best-effort): 28
Sample missing lines (up to 10):
- --- PAGE 1 ---
- Vijay Kumar was born on March 15, 1989, in Jaipur, Rajasthan, making him 35 years old as of 2024.
- His birthdate is formatted as 1989-03-15 in ISO format for easy parsing, while his age serves as a
- key demographic marker for analytical purposes. Born and raised in the Pink City of India, his
- birthplace provides valuable regional profiling context, and his O+ blood group is noted for
- emergency contact purposes. As an Indian national, his citizenship status is important for
- Vijay's professional journey began on July 1, 2012, when he joined his first company as a Junior
- Developer with an annual salary of 350,000 INR. His career progression shows remarkable growth,
- with his current role at Resse Analytics beginning on June 15, 2021, where he serves as a Senior
- Data Engineer earning 2,800,000 INR annually. Before this position, h

In [21]:
##Step : 8
##Convert the LLM JSON into an Excel (Output.xlsx)
import pandas as pd

# Normalize list of dicts to dataframe and preserve order
df = pd.DataFrame(extracted_list)
# Ensure columns Key, Value, Comments exist
for c in ["Key","Value","Comments"]:
    if c not in df.columns:
        df[c] = ""

# Save to excel
output_file = "Output.xlsx"
df.to_excel(output_file, index=False, engine="openpyxl")
print("Saved", output_file)

# Show head
df.head(20)


Saved Output.xlsx


,Key,Value,Comments
0,Full Name,Vijay Kumar,
1,Date of Birth,1989-03-15,"Vijay Kumar was born on March 15, 1989, in Jai..."
2,Age,35,making him 35 years old as of 2024.
3,Birthplace,"Jaipur, Rajasthan",Born and raised in the Pink City of India.
4,Blood Group,O+,his O+ blood group is noted for emergency cont...
5,Citizenship,Indian,"As an Indian national, his citizenship status ..."
6,First Job Start Date,2012-07-01,"Vijay's professional journey began on July 1, ..."
7,First Job Title,Junior Developer,when he joined his first company as a Junior D...
8,First Job Salary,"350,000 INR","with an annual salary of 350,000 INR."
9,Current Job Start Date,2021-06-15,His career progression shows remarkable growth...


In [22]:
from google.colab import files
files.download("Output.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
##Save code files for GitHub and create README
# Save main.py
main_py = """
# main.py - extractor script (skeleton)
# (You can adapt this from the Colab cells)
"""
with open("main.py", "w") as f:
    f.write(main_py)

readme = """
# PDF → Structured Excel Extractor
Instructions...
"""
with open("README.md", "w") as f:
    f.write(readme)

print("Saved main.py and README.md; you can push these to GitHub.")


Saved main.py and README.md; you can push these to GitHub.
